# SQL Queries

## Total Number of Events

In [ ]:
%%sql

SELECT COUNT(*) AS count
FROM events;

## Events by Region

In [ ]:
%%sql

SELECT country, COUNT(country) AS count
FROM events
GROUP BY 1
ORDER BY count DESC
LIMIT 10;

## Events by Top 5 Advertisers

In [ ]:
%%sql

SELECT advertiser, COUNT(*) AS count
FROM events
WHERE (advertiser LIKE '%Subway%' OR advertiser LIKE '%McDonals%' OR advertiser LIKE '%Starbucks%' OR advertiser LIKE '%Dollar General%' OR advertiser LIKE '%YUM! Brands%' OR advertiser LIKE '%Dunkin Brands Group%')
GROUP BY 1
ORDER BY count DESC;

## Ad Visitors by Gender and Income

In [ ]:
%%sql

SELECT *
FROM (SELECT *, DENSE_RANK() OVER (ORDER BY xx.z___min_rank) AS z___pivot_row_rank, RANK() OVER (PARTITION BY xx.z__pivot_col_rank ORDER BY xx.z___min_rank) AS z__pivot_col_ordering, CASE
        WHEN xx.z___min_rank = xx.z___rank THEN 1
        ELSE 0
      END AS z__is_highest_ranked_cell
    FROM (SELECT *, Min(aa.z___rank) OVER (PARTITION BY aa.`events.income`) AS z___min_rank
        FROM (SELECT *, RANK() OVER (ORDER BY CASE
                WHEN bb.z__pivot_col_rank = 1 THEN (CASE
                    WHEN bb.`events.count` IS NOT NULL THEN 0
                    ELSE 1
                  END)
                ELSE 2
              END, CASE
                WHEN bb.z__pivot_col_rank = 1 THEN bb.`events.count`
                ELSE NULL
              END DESC, bb.`events.count` DESC, bb.z__pivot_col_rank, bb.`events.income`) AS z___rank
            FROM (SELECT *, DENSE_RANK() OVER (ORDER BY CASE
                    WHEN ww.`events.gender` IS NULL THEN 1
                    ELSE 0
                  END, ww.`events.gender`) AS z__pivot_col_rank
                FROM (SELECT events.gender AS `events.gender`, events.income AS `events.income`, COUNT(*) AS `events.count`
                    FROM adtech.events AS events
                    WHERE (events.income <> 'unknown' OR events.income IS NULL)
                    GROUP BY 1, 2) ww) bb
            WHERE bb.z__pivot_col_rank <= 16384) aa) xx) zz
WHERE (zz.z__pivot_col_rank <= 50 OR zz.z__is_highest_ranked_cell = 1) AND (zz.z___pivot_row_rank <= 500 OR zz.z__pivot_col_ordering = 1)
ORDER BY zz.z___pivot_row_rank;

# LangChain and OpenAI

In [ ]:
!pip install --upgrade langchain==0.1 --quiet
!pip install --upgrade openai==1.7.1 --quiet

In [ ]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

In [ ]:
from langchain import OpenAI, SQLDatabase
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.agents import create_sql_agent

In [ ]:
db = SQLDatabase.from_uri(connection_url)

llm = OpenAI(
    model_name = "gpt-3.5-turbo-instruct",
    temperature = 0,
    verbose = False
)

toolkit = SQLDatabaseToolkit(db = db, llm = llm)

agent_executor = create_sql_agent(
    llm = OpenAI(
        model_name = "gpt-3.5-turbo-instruct",
        temperature = 0
    ),
    toolkit = toolkit,
    verbose = False
)

In [ ]:
query = input("Please enter your question:")
agent_executor.run(query)